# **COVID-19 Data Analysis & Visualization**
![Image](https://www.furman.edu/covid-19/wp-content/uploads/sites/177/2020/03/CoronaVirusHeader-Final-3.jpg)


**Content:**
1. [Load Libraries and Data](#1)
2. [Confirmed Cases Worldwide](#2)
3. [Worldwide Daily Cases and Deaths](#3)
4. [Growth Rate](#4)
5. [World Map: Confirmed Cases and Fatalities Growth](#5)
6. [Trend of COVID-19 in Top 20 Affected Countries](#6)
7. [Mortality Rate](#7)
 * [20 Countries with the Highest Mortality Rate](#8)
 * [20 Countries with the Lowest Mortality Rate](#9)
 * [Mortality Rate Trend of COVID-19 in Top 20 Affected Countries](#22)
8. [COVID-19 in Turkey](#10)
 * [Healing Rate](#11)
 * [Mortality Rate](#24)
9. [Demographic Data of the Countries and the Covid-19 Pandemic](#12)
 *  [Cleaning Data](#13)
 *  [Correlation Map](#14)
10. [Data Analysis of Confirmed Cases in the State of Ontario](#16)
 *  [Gender Distribution of Cases](#17)
 *  [Age Groups Distribution of Cases](#18)
 *  [Age-Gender Distribution of Cases](#19)
 *  [Coronavirus Acquisition-Related Information](#20)
11. [Final](#21)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install pycountry_convert

<a id = "1"></a><br>
**Load Libraries and Data**

In [ ]:
import json
# datetime oprations
from datetime import timedelta
# to get web contents
from urllib.request import urlopen
# basic visualization package
import matplotlib.pyplot as plt
# advanced ploting
import seaborn as sns
 
import folium
import branca
import datetime as dt
import requests
import sys
from itertools import chain

# plotly
from plotly import tools, subplots
import pycountry
import pycountry_convert as pc

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "plotly_dark"
%matplotlib inline

# for offline ploting
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)

# hide warnings
import warnings
warnings.filterwarnings('ignore')

# models 
from sklearn.preprocessing import OrdinalEncoder
from sklearn import metrics
import xgboost as xgb
from xgboost import XGBRegressor
from xgboost import plot_importance, plot_tree
from sklearn.model_selection import GridSearchCV


In [ ]:
df = pd.read_csv('../input/novel-corona-virus-2019-dataset/covid_19_data.csv')

In [ ]:
#verileri düzeltme
df.rename(columns={'ObservationDate':'Date','Province/State':'Province_State',
                   'Country/Region':'Country_Region','Confirmed':'ConfirmedCases',
                   'Deaths':'Fatalities'},inplace=True)
df.loc[df['Country_Region']=='Mainland China','Country_Region']='China'
df['Date'] = pd.to_datetime(df['Date'],format='%m/%d/%Y')
df['Day'] = df.Date.dt.dayofyear
df['cases_lag'] = df.groupby(['Country_Region','Province_State'])['ConfirmedCases'].shift(1) #önceki günün vaka sayısı
df['deaths_lag'] = df.groupby(['Country_Region','Province_State'])['Fatalities'].shift(1) #önceki günün ölüm sayısı
df['Daily Cases'] = df['ConfirmedCases'] - df['cases_lag']  
df['Daily Deaths'] = df['Fatalities'] - df['deaths_lag']         

In [ ]:
df.head()

In [ ]:
class country_utils():
    def __init__(self):
        self.d = {}
    
    def get_dic(self):
        return self.d
    
    def get_country_details(self,country):
        """Returns country code(alpha_3) and continent"""
        try:
            country_obj = pycountry.countries.get(name=country)
            if country_obj is None:
                c = pycountry.countries.search_fuzzy(country)
                country_obj = c[0]
            continent_code = pc.country_alpha2_to_continent_code(country_obj.alpha_2)
            continent = pc.convert_continent_code_to_continent_name(continent_code)
            return country_obj.alpha_3, continent
        except:
            if 'Congo' in country:
                country = 'Congo'
            elif country == 'Diamond Princess' or country == 'Laos' or country == 'MS Zaandam'\
            or country == 'Holy See' or country == 'Timor-Leste':
                return country, country
            elif country == 'Korea, South' or country == 'South Korea':
                country = 'Korea, Republic of'
            elif country == 'Taiwan*':
                country = 'Taiwan'
            elif country == 'Burma':
                country = 'Myanmar'
            elif country == 'West Bank and Gaza':
                country = 'Gaza'
            else:
                return country, country
            country_obj = pycountry.countries.search_fuzzy(country)
            continent_code = pc.country_alpha2_to_continent_code(country_obj[0].alpha_2)
            continent = pc.convert_continent_code_to_continent_name(continent_code)
            return country_obj[0].alpha_3, continent
    
    def get_iso3(self, country):
        return self.d[country]['code']
    
    def get_continent(self,country):
        return self.d[country]['continent']
    
    def add_values(self,country):
        self.d[country] = {}
        self.d[country]['code'],self.d[country]['continent'] = self.get_country_details(country)
    
    def iso3(self,country):
        if country in self.d.keys():
            return self.get_iso3(country)
        else:
            self.add_values(country)
            return self.get_iso3(country)
        
    def fetch_continent(self,country):
        if country in self.d.keys():
            return self.get_continent(country)
        else:
            self.add_values(country)
            return self.get_continent(country)

<a id = "2"></a><br>
# **Confirmed Cases Worldwide**

In [ ]:
#df.ConfirmedCases = np.abs(df.ConfirmedCases) #negatif bir değer varsa onu düzeltiyoruz
df1 = df.copy() #veri setini kopyalıyoruz
date = df1.Date.max() #en güncel tarihi alıyoruz
df1 = df1[df1['Date']==date]
obj = country_utils()
df1.Province_State.fillna('',inplace=True) #NA,NaN değerlerini boşlukla dolduruyoruz
df1['continent'] = df1.apply(lambda x: obj.fetch_continent(x['Country_Region']), axis=1) #ülkeleri kıtalara ayırıyoruz
df1["world"] = "World" # hepsini kapsayan tek bir kök düğüm
fig = px.treemap(df1, path=['world', 'continent', 'Country_Region'], values='ConfirmedCases',
                  color='ConfirmedCases', hover_data=['Country_Region'],
                  color_continuous_scale='BuPu', title='COVID 19 Vakalarının Dünya Çapında Dağılımı')
fig.update_layout(width=750,template='seaborn')
fig.show()

In [ ]:
def daily_measures(df): #günlük ölçümler
    df.loc[0,'Daily Cases'] = df.loc[0,'ConfirmedCases']
    df.loc[0,'Daily Deaths'] = df.loc[0,'Fatalities']
    for i in range(1,len(df)):
        df.loc[i,'Daily Cases'] = df.loc[i,'ConfirmedCases'] - df.loc[i-1,'ConfirmedCases']  #Günlük vaka sayılarını bulmak için o günkü toplam vaka sayısından önceki günün onaylanan vaka sayısını çıkarıyoruz.
        df.loc[i,'Daily Deaths'] = df.loc[i,'Fatalities'] - df.loc[i-1,'Fatalities']
    df.loc[0,'Daily Cases'] = 0 #Daha önceki vaka sayısını günlük olarak bilmediğimiz için ilk satırdaki değerleri 0 a eşitliyoruz.
    df.loc[0,'Daily Deaths'] = 0
    return df

In [ ]:
dfWorld = df.copy()
dfWorld = dfWorld.groupby('Date',as_index=False)['ConfirmedCases','Fatalities','Daily Cases','Daily Deaths'].sum() #Dünyadaki tüm vaka sayılarını ve ölümleri tarihe göre (günlük) grupluyoruz.
dfWorld = daily_measures(dfWorld) #dünyadaki günlük vaka sayılarını alıyoruz
dfWorld['Cases:5-Day Moving Average'] = dfWorld['Daily Cases'].rolling(5).mean() #Vaka sayısının 5 günlük hareketli ortalamasını alıyoruz
dfWorld['Deaths:5-Day Moving Average'] = dfWorld['Daily Deaths'].rolling(5).mean() #Ölüm sayısının 5 günlük hareketli ortalamasını alıyoruz

In [ ]:
dfWorld.head(15)

<a id = "3"></a><br>
# **Worldwide Daily Cases and Deaths**

In [ ]:
fig2 = go.Figure(data=[
    go.Bar(name='Cases', x=dfWorld['Date'], y=dfWorld['Daily Cases']),
    go.Bar(name='Deaths', x=dfWorld['Date'], y=dfWorld['Daily Deaths'])
])
fig2.show()

In [ ]:
#Hareketli ortalama çizgimizi ekleyiyoruz
fig2.add_trace(go.Scatter(name='Cases:5-Day Moving Average',x=dfWorld['Date'],y=dfWorld['Cases:5-Day Moving Average'],marker_color='white'))
fig2.add_trace(go.Scatter(name='Deaths:5-Day Moving Average',x=dfWorld['Date'],y=dfWorld['Deaths:5-Day Moving Average'],marker_color='magenta'))
#Grafiği daha okunabilir hale getirme
fig2.update_layout(barmode='overlay', title='Dünya Çapında Günlük Vaka ve Ölüm Sayısı',hovermode='x',
                template='plotly_dark', width=780,xaxis=dict(mirror=True,linewidth=1,linecolor='white',showgrid=False),
                 yaxis=dict(mirror=True,linewidth=1,linecolor='aliceblue'),legend=dict(orientation='h',x=0.1,y=-0.2))
fig2.show()

<a id = "4"></a><br>
# **Growth Rate (Büyüme Oranı)**
Büyüme Oranı, o günkü yeni vaka sayısının, önceki günkü yeni vaka sayısına bölünmesiyle hesaplanmaktadır. Büyüme Oranı 1'den fazla olduğunda yeni vaka sayısı artmış, 1'den düşük olduğunda yeni vaka sayısı düşmüş olacaktır.

Aşağıdaki grafikte görüldüğü üzere Dünya'da ki "COVID-19 Büyüme Oranı", Nisan ayından itibaren 1 civarında seyretmektedir. Bu durum Nisan ayından itibaren onaylanan yeni vaka sayılarının, artış göstermediği fakat aynı zamanda azalış da göstermediği anlamına gelmektedir.

In [ ]:
dfWorld['growthFactor'] = dfWorld['Daily Cases'] / dfWorld['Daily Cases'].shift(1)
dfWorld.tail() #Feature Engineering

In [ ]:
fig = px.line(dfWorld, x='Date', y='growthFactor', title='Dünya Çapında COVID-19 Vakalarının Büyüme Oranı')
fig.add_trace(go.Scatter(x=[dfWorld['Date'].min(), dfWorld['Date'].max()], y=[1., 1.], name='Büyüme Oranı=1.', line=dict(dash='dash', color=('rgb(237, 37, 75)'))))
fig.update_layout(hovermode='closest',template='plotly_dark',width=750,xaxis=dict(mirror=True,linewidth=1,linecolor='white'),
                 yaxis=dict(mirror=True,linewidth=1,linecolor='white'))
fig.update_yaxes(range=[0., 7.])
fig.show()

<a id = "5"></a><br>
# **World Map: Confirmed Cases and Fatalities Growth (Logarithmic Scale)**

In [ ]:
dfWmap = df.copy()
dfWmap['Date']=dfWmap['Date'].astype(str)
dfWmap = dfWmap.groupby(['Date','Country_Region'], as_index=False)['ConfirmedCases','Fatalities'].sum()
dfWmap.head(10)

In [ ]:
dfWmap['iso_alpha'] = dfWmap.apply(lambda x: obj.iso3(x['Country_Region']), axis=1)
dfWmap.head()

In [ ]:
#Veriler çok geniş bir değer aralığında (örn: ABD ve Yeni Zelanda'nın vaka sayısı arasındaki fark) olduğu için logaritmik ölçek kullanıyoruz.
dfWmap['log(ConfirmedCases)'] = np.log(dfWmap.ConfirmedCases + 1)
dfWmap['log(Fatalities)'] = np.log(dfWmap.Fatalities + 1)

In [ ]:
px.choropleth(dfWmap, #Confirmed Cases Growth
              locations="iso_alpha", 
              color="log(ConfirmedCases)", 
              hover_name="Country_Region", 
              hover_data=["ConfirmedCases"] ,
              animation_frame="Date",
              color_continuous_scale=px.colors.sequential.BuPu, 
              title='Logaritmik Ölçekte Ülkelere Göre Onaylanan Vaka Sayısı Büyümesi')

In [ ]:
px.choropleth(dfWmap, #Fatalities Growth
             locations='iso_alpha',
             color='log(Fatalities)',
             hover_name='Country_Region',
             hover_data=["Fatalities"],
             animation_frame="Date",
             color_continuous_scale=px.colors.sequential.amp,
             title='Logaritmik Ölçekte Ülkelere Göre Ölüm Sayısının Gelişimi')

<a id = "6"></a><br>
# **Trend of COVID-19 in Top 20 Affected Countries**

In [ ]:
last_date = df.Date.max() #en güncel toplam vaka saysını alabilmek için en güncel tarihi alıyoruz
dfCountries = df[df['Date']==last_date]
dfCountries = dfCountries.groupby('Country_Region', as_index=False)['ConfirmedCases','Fatalities'].sum()
dfCountries = dfCountries.nlargest(20,'ConfirmedCases') #En çok vaka sayısı olan 20 ülkeyi alıyoruz
#En çok etkilenen 20 ülkenin trendi
df_trend = df.groupby(['Date','Country_Region'], as_index=False)['ConfirmedCases','Fatalities'].sum()
df_trend = df_trend.merge(dfCountries, on='Country_Region')
df_trend.drop(['ConfirmedCases_y','Fatalities_y'],axis=1, inplace=True)
df_trend.rename(columns={'Country_Region':'Country', 'ConfirmedCases_x':'Cases', 'Fatalities_x':'Deaths'}, inplace=True)

In [ ]:
fig = px.line(df_trend, x='Date', y='Cases', color='Country', title='En Çok Etkilnenen 20 Ülkenin COVID-19 Vaka Sayısı Gelişimi')
fig.update_layout(hovermode='closest',template='plotly_dark',width=750,xaxis=dict(mirror=True,linewidth=1,linecolor='white',showgrid=False),
                 yaxis=dict(mirror=True,linewidth=2,linecolor='white'))
fig.show() #total confirmed cases growth

In [ ]:
fig = px.line(df_trend, x='Date', y='Deaths', color='Country', title='En Çok Etkilnenen 20 Ülkenin COVID-19 Ölüm Sayısı Gelişimi')
fig.update_layout(hovermode='closest',template='plotly_dark',width=750,xaxis=dict(mirror=True,linewidth=1,linecolor='white',showgrid=False),
                 yaxis=dict(mirror=True,linewidth=2,linecolor='white'))
fig.show() #total deaths growth

<a id = "7"></a><br>
# **Mortality Rate (Ölüm Oranı)**
Toplam ölüm sayılarının, onaylanan vaka sayılarına bölünmesiyle hesaplanmaktadır.

In [ ]:
dfWmap['MortalityRate%'] = round((dfWmap.Fatalities/dfWmap.ConfirmedCases)*100,1)
dfWmap.tail()

In [ ]:
px.choropleth(dfWmap, 
                    locations="iso_alpha", 
                    color="MortalityRate%", 
                    hover_name="Country_Region",
                    hover_data=["ConfirmedCases","Fatalities"],
                    animation_frame="Date",
                    color_continuous_scale=px.colors.sequential.OrRd,
                    title = 'Dünyada Günlük Ölüm Oranlarının Değişimi')

In [ ]:
last_date = df.Date.max()
dfMor= df[df['Date']==last_date]
dfMor=dfMor.groupby(['Country_Region'], as_index=False)['Country_Region','ConfirmedCases','Fatalities'].sum()
dfMor['MortalityRate%'] = round((dfMor.Fatalities/dfMor.ConfirmedCases)*100,1)
dfMorMax = dfMor.nlargest(20,'MortalityRate%')
dfMorMax

<a id = "8"></a><br>
**20 Countries with the Highest Mortality Rate**

In [ ]:
fig = px.bar(dfMorMax[:20].iloc[::-1],
            x='MortalityRate%',
            y='Country_Region',
            title="Ölüm Oranı En Yüksek 20 Ülke (20 Countries with the Highest Mortality Rate)",
            text='MortalityRate%',
            height=750,
            orientation='h')
fig.show()

<a id = "9"></a><br>
**20 Countries with the Lowest Mortality Rate**

In [ ]:
dfMorMin = dfMor.nsmallest(20,'MortalityRate%')
fig = px.bar(dfMorMin[:20].iloc[::-1],
            x='MortalityRate%',
            y='Country_Region',
            title="Ölüm Oranı En Düşük 20 Ülke (20 Countries with the Lowest Mortality Rate)",
            text='MortalityRate%',
            height=750,
            orientation='h')
fig.show()

In [ ]:
df_trend['MortalityRate%'] = round((df_trend.Deaths/df_trend.Cases)*100,1)

<a id = "22"></a><br>
**Mortality Rate Trend of COVID-19 in Top 20 Affected Countries**

In [ ]:
fig = px.line(df_trend, x='Date', y='MortalityRate%', color='Country', title='COVID-19 dan En Çok Etkilenen 20 Ülkenin Ölüm Oranı Değişimleri')
fig.update_layout(hovermode='closest',template='plotly_dark',xaxis=dict(mirror=True,linewidth=1,linecolor='white',showgrid=False),
                 yaxis=dict(mirror=True,linewidth=1,linecolor='white'))
fig.show()

<a id = "10"></a><br>
# **COVID-19 in TURKEY**

In [ ]:
dfTr = df[df['Country_Region']=='Turkey']
dfTr.tail()

In [ ]:
dfTR = dfTr.groupby(['Date'], as_index=False)['ConfirmedCases','Fatalities','Recovered'].sum()

In [ ]:
melt_df_tr = pd.melt(dfTR, id_vars=['Date'], value_vars=['ConfirmedCases', 'Fatalities', 'Recovered'])
melt_df_tr

In [ ]:
fig = px.line(melt_df_tr, x="Date", y="value", color='variable', 
              title="Türkiye'de Onaylanan Vakalarının, Ölümlerin ve İyileşen Hastaların İlerleyişi")
fig.show()

In [ ]:
dfTR['casesPrev'] = dfTR['ConfirmedCases'].shift(1)
dfTR['casesDaily']= dfTR['ConfirmedCases'] - dfTR['casesPrev'] #günlük vaka sayısı
dfTR.tail()

In [ ]:
dfTR['deathsDaily']= dfTR['Fatalities'] - dfTR['Fatalities'].shift(1) #günlük ölüm sayısı

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=dfTR['Date'], y=dfTR['casesDaily'],
                    mode='lines+markers',
                    name='Günlük Vaka Sayıları'))

fig.add_trace(go.Scatter(x=dfTR['Date'], y=dfTR['deathsDaily'],
                    mode='lines+markers',
                    name='Günlük Ölüm Sayıları'))

fig.update_layout(barmode='overlay', title="Türkiye'de Günlük Vaka ve Ölüm Sayısı",hovermode='x',
                template='plotly_dark',xaxis=dict(mirror=True,linewidth=1,linecolor='aliceblue',showgrid=False),
                 yaxis=dict(mirror=True,linewidth=1,linecolor='aliceblue'),legend=dict(orientation='h',x=0.2,y=-0.2))
fig.show()

In [ ]:
dfTR['growthRate'] = dfTR['casesDaily'] / dfTR['casesDaily'].shift(1)
fig = px.line(dfTR, x='Date', y='growthRate', title="Türkiye'de COVID-19 Vakalarının Büyüme Oranı")
fig.add_trace(go.Scatter(x=[dfWorld['Date'].min(), dfWorld['Date'].max()], y=[1., 1.], name='Büyüme Oranı=1.', line=dict(dash='dash', color=('rgb(237, 37, 75)'))))
fig.update_layout(hovermode='closest',template='plotly_dark',xaxis=dict(mirror=True,linewidth=1,linecolor='white',showgrid=False),
                 yaxis=dict(mirror=True,linewidth=1,linecolor='white'))
fig.update_yaxes(range=[0., 4.])
fig.show()

<a id = "11"></a><br>
# **Healing Rate (Vakaların İyileşme Oranı)**

Toplam iyileşen hasta sayısının, vaka sayısına bölünmesiyle hesaplanmaktadır.

In [ ]:
dfTR['healingRate%'] = round((dfTR.Recovered/dfTR.ConfirmedCases)*100,1)
dfTR

In [ ]:
fig = px.line(dfTR, x='Date', y='healingRate%', title="Türkiye'de COVID-19 Vakalarının İyileşme Oranı")
fig.update_layout(hovermode='closest',template='plotly_dark',width=750, xaxis=dict(mirror=True,linewidth=1,linecolor='white',showgrid=False),
                 yaxis=dict(mirror=True,linewidth=1,linecolor='white'))
fig.show()

<a id = "24"></a><br>
**Mortality Rate (Turkey)**

In [ ]:
dfTR['mortalityRate%'] = round((dfTR.Fatalities/dfTR.ConfirmedCases)*100,1)
fig = px.line(dfTR, x='Date', y='mortalityRate%', title="Türkiye'de COVID-19 Vakalarının Ölüm Oranı")
fig.update_layout(hovermode='closest',template='plotly_dark',width=750, xaxis=dict(mirror=True,linewidth=1,linecolor='white',showgrid=False),
                 yaxis=dict(mirror=True,linewidth=1,linecolor='white'))
fig.show()

<a id = "12"></a><br>
# **Ülkelerin Demografik Verilerinin Covid-19 Pandemi Süreciyle Olan İlişkisi**
**The Relationship between the Demographic Data of the Countries and the Covid-19 Pandemic**

In [ ]:
population = pd.read_csv('../input/population-by-country-2020/population_by_country_2020.csv')
population.head()

In [ ]:
pop = population[['Country (or dependency)','Population (2020)', 'Med. Age','Density (P/Km²)']]
pop.columns= ['country','population','avrgAge','density']
pop.head(10)

In [ ]:
pop.loc[pop['country'] == 'United States', "country"] = "US"
pop.loc[pop['country'] == 'Czech Republic (Czechia)', "country"] = 'Czech Republic'
pop.loc[pop['country'] == 'United Kingdom', "country"] = "UK"
pop.head()

In [ ]:
hc1 = pd.read_html('https://www.numbeo.com/health-care/rankings_by_country.jsp')
hc1

In [ ]:
hc = pd.read_html('https://www.numbeo.com/health-care/rankings_by_country.jsp')
hc = hc[['Country', 'Health Care Index']]

In [ ]:
hc.columns= ['country','healthcare_index']
hc.loc[hc['country'] == 'United States', "country"] = "US"
hc.loc[hc['country'] == 'United Kingdom', "country"] = "UK"
hc.tail()

In [ ]:
last_date = df.Date.max()
dfTotal = df[df['Date']==last_date]
dfTotal=dfTotal.groupby(['Country_Region'], as_index=False)['Country_Region','ConfirmedCases','Fatalities','Recovered'].sum()
dfTotal['mortalityRate%'] = round((dfTotal.Fatalities/dfTotal.ConfirmedCases)*100,1)
dfTotal['healingRate%'] = round((dfTotal.Recovered/dfTotal.ConfirmedCases)*100,1)
dfTotal.rename(columns={'Country_Region':'country'},inplace=True)
dfTotal.head()

In [ ]:
dfTotal = pd.merge(dfTotal, pop, on='country', how='left',copy = False) #3 tane veri setini birleştiriyoruz
dfTotal = pd.merge(dfTotal, hc, on='country', how = 'left',copy = False)
dfTotal

<a id = "13"></a><br>
**Cleaning Data (Veri Temizleme)**

In [ ]:
dfTotal.info() #kaç tane null veri var bakalım

In [ ]:
dfTotal[dfTotal["avrgAge"].isnull()] #avrgAge sütunu NaN olan kayıtları listeliyoruz

In [ ]:
dfTotal = dfTotal.dropna(how='any', subset=['population', 'density','avrgAge'])
#population,density,avrgAge,healthcare_index sütunlarındaki verilerin hepsi NaN olduğu için,
#ortalamayla doldurmak yerine bu kayıtları silmeyi tercih ettim

In [ ]:
assert dfTotal["population"].notnull().all() #kontrol ediyoruz

In [ ]:
dfTotal #görüldüğü üzere 12 kayıt silinmiş 178 satır var

In [ ]:
dfTotal['avrgAge'] = dfTotal['avrgAge'].replace('N.A.', '0') #4.satırda görüldüğü üzere avrgAge sütununda N.A şeklinde değerler var
dfTotal['avrgAge'] = dfTotal['avrgAge'].astype('int') #bu değerleri 0'a eşitleyip string tipi int yaptık
dfTotal['avrgAge'] = dfTotal['avrgAge'].replace(0, dfTotal['avrgAge'].mean()) #ve bu kayıp verileri sütun ortalamasıyla doldurduruyoruz
dfTotal['healthcare_index'] = dfTotal['healthcare_index'].replace(np.nan, dfTotal['healthcare_index'].mean())

dfTotal['avrgAge'] = dfTotal['avrgAge'].astype('int')
dfTotal['ConfirmedCases'] = dfTotal['ConfirmedCases'].astype('int')
dfTotal['Fatalities'] = dfTotal['Fatalities'].astype('int')
dfTotal['Recovered'] = dfTotal['Recovered'].astype('int')
dfTotal['population'] = dfTotal['population'].astype('int')
dfTotal['density'] = dfTotal['density'].astype('int') #float tipindeki sütunları int veri tipine dönüştürdük

dfTotal.info()
dfTotal.head()
#verileri temizledik

<a id = "14"></a><br>
**Correlation Map (Korelasyon Haritası)**

Özellikler arasındaki ilişkiyi (bağlılığı) renkli tablo şeklinde görüyoruz. Görüldüğü üzere onaylanan vaka sayısı (ConfirmedCases) ve gerçekleşen ölümler (Fatalities) arasında pozitif doğrusal bir ilişki olduğu görülüyor. Aynı şekilde ülke nüfusu (population) ve onaylanan pozitif vakalar (ConfirmedCases) arasında da pozitif doğrusal bir ilişki olduğu söylenebilir.

In [ ]:
f,ax = plt.subplots(figsize=(15, 15))
sns.heatmap(dfTotal.corr(), annot=True, linewidths=.5, fmt= '.1f',ax=ax)
plt.show()

In [ ]:
fig = px.scatter(dfTotal, x='population', y='ConfirmedCases', color='mortalityRate%',width=770, height=590, 
                 hover_name='country', log_x=True, log_y=True, title='Population vs Confirmed (Nüfus - Onayalanan Vaka)',template='ggplot2', range_color=[0,12])
fig.show()

In [ ]:
fig = px.scatter(dfTotal, x='Recovered', y='Fatalities', color='healthcare_index',width=770, height=590, 
                 hover_name='country', log_x=True, log_y=True, title='Population vs Fatalities (Nüfus - Ölümler)',template='ggplot2', range_color=[50,85])
fig.show()

In [ ]:
fig = px.scatter(dfTotal, x='healthcare_index', y='avrgAge', color='healingRate%',width=770, height=590,
                 hover_name='country', log_x=True, log_y=True, title='Healthcare Index vs Avrrage Age (Sağlık Hizmeti - Ortalama Yaş)',template='ggplot2')
fig.show()

In [ ]:
fig = px.scatter(dfTotal, x='ConfirmedCases', y='density', color='healthcare_index',width=770, height=590, 
                 hover_name='country', log_x=True, log_y=True, title='Density(pop./km2) vs Confirmed Casses (Nüfus Yoğunluğu - Onayalanan Vaka)',template='ggplot2')
fig.show()

In [ ]:
fig = px.scatter(dfTotal, x='mortalityRate%', y='Fatalities', color='healthcare_index',width=770, height=590, 
                 hover_name='country', log_x=True, log_y=True, title='Fatalities vs Mortality Rate(%) (Ölümler-Ölüm Oranı)',template='ggplot2', range_y=[5,40000],range_color=[50,85])
fig.show()

<a id = "16"></a><br>
# **Data Analysis of Confirmed Cases in the State of Ontario**
**Ontario Eyaletindeki Onaylanan Vakaların Veri Analizi**

In [ ]:
dfCases = pd.read_csv('../input/uncover/UNCOVER/ontario_government/confirmed-positive-cases-of-covid-19-in-ontario.csv')
dfCases.head()

In [ ]:
df_cases = dfCases.groupby(['age_group','client_gender','case_acquisitioninfo'], as_index=False)['row_id'].count()
df5 = dfCases.groupby(['age_group','client_gender'], as_index=False)['row_id'].count()
dfCases = dfCases[['age_group','client_gender', 'case_acquisitioninfo']]
dfCases.info()

<a id = "17"></a><br>
**Gender Distribution of Cases** (Vakaların Cinsiyete Göre Dağılımının Dairesel Grafikte Gösterimi)****

In [ ]:
Male =len(dfCases[dfCases['client_gender'] == "MALE"])
Female = len(dfCases[dfCases['client_gender']== "FEMALE"])
plt.figure(figsize=(12,8))

#Vakaların cinsiyet 
labels = 'Male','Female'
sizes = [Male,Female]
colors = ['skyblue', 'gold']
explode = (0, 0)  
 
# Plot
plt.pie(sizes, explode=explode, labels=labels, colors=colors,
autopct='%1.1f%%', shadow=True, startangle=50)
 
plt.axis('equal')
plt.show()

In [ ]:
df_cases

<a id = "18"></a><br>
**Age Groups Distribution of Cases** (Vakaların Yaş Gruplarına Göre Dağılımı)

In [ ]:
df5['age_group'] = df5['age_group'].replace('Unknown',np.nan)  #veri temizleme
df6 = df5.dropna(how='any', subset=['age_group'])
df6['client_gender'] = df6['client_gender'].replace(['UNKNOWN','(blank)','TRANSGENDER','OTHER'],np.nan)
df6=df6.dropna(how='any', subset=['client_gender'])
df6

In [ ]:
fig = go.Figure()  
dfAges = df6.groupby('age_group',as_index=False)['row_id'].sum()  
fig.add_trace(go.Pie(labels=dfAges['age_group'], values=dfAges['row_id'], hole=0.3))
fig.update_layout(hovermode='closest',title="Vakaların Yaş Gruplarına Göre Dağılımı", template='ggplot2',width=750)
fig.show()

In [ ]:
dfMales = df6[df6['client_gender']=='MALE'].groupby('age_group',as_index=False)['row_id'].sum()
dfFemales = df6[df6['client_gender']=='FEMALE'].groupby('age_group',as_index=False)['row_id'].sum()

<a id = "19"></a><br>
**Age-Gender Distribution of Cases** (Vakaların Yaşa ve Cinsiyete Göre Dağılımı)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(name='Males', x=dfMales['age_group'], y=dfMales['row_id']))
fig.add_trace(go.Bar(name='Females', x=dfFemales['age_group'], y=dfFemales['row_id']))
fig.update_layout(title="Vakaların Yaşa ve Cinsiyete Göre Dağılımı (Age-Gender Distribution of Cases)")
fig.show()

In [ ]:
dfCases['case_acquisitioninfo'].unique()

<a id = "20"></a><br>
**Coronavirus Acquisition-Related Information (Hastaların Virüsü Edinme Yolları)**
* Travel-Related (Seyehat)
* Contact of a confirmed case (Onaylanan bir vaka ile temas)
* Neither (Diğer)
* Information pending (Bilgi Bekleniyor)

In [ ]:
caseInfo  = df_cases.groupby('case_acquisitioninfo',as_index=False)['row_id'].sum() 
fig = px.bar(caseInfo,
            x='row_id',
            y='case_acquisitioninfo',
            title="Virüsü Edinme Yolları(Coronavirus Acquisition-Related Information)",
            text='row_id',
            orientation='h',
            width=750)
fig.show()

<a id = "21"></a><br>
 > Thank you for reading my kernel. I hope, we will be completely back to our usual daily life as soon as possible and we'll see better days.

**Betül Sena Çağlar**

**Contact:**
**[LinkedIn](https://www.linkedin.com/in/betül-sena-çağlar-783307188)**